<a href="https://colab.research.google.com/github/Maximo-Rulli/dynamic-steps-dlm/blob/main/blocks-entropy-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing the entropy of Diffusion blocks by DLMs

### Essential imports

In [ ]:
#Essential imports
import torch
import numpy as np
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModel

#Repository's functions
from MMaDA.models import MMadaModelLM

ModuleNotFoundError: No module named 'models'

In [ ]:
!python MMaDA/generate.py

python: can't open file '/home/maxrul/dev/dynamic-steps-dlm/generate.py': [Errno 2] No such file or directory
